In [23]:
import gym
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from PIL import Image
import time

import torch
import torch.nn as nn
import torchvision.transforms as T
import torch.optim as optim
import torch.nn.functional as F

In [24]:
env = gym.make('CartPole-v1').unwrapped

In [25]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
    
print(device)

cuda


In [26]:
# Replay Memory
Transition = namedtuple('Transition', ('state','action', 'next_state', 'reward'))
class replay(object):
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self, *args):
        if len(self.memory)<self.capacity:
            self.memory.append()
        self.memory[self.position] = Transition(*args)
        self.position= (self.position+1)%self.capacity
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [27]:
class DQN(nn.Module):
    
    def __init__(self, in_channels = 3, num_actions = 2):
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_sizes = 5, stride = 2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.fc4 = nn.Linear(7*7*32, 512)
        self.fc4 = nn.Linear(512, num_actions)
        
    def forward(sefl, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.bn1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.bn2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.bn3(x))
        x = F.relu(self.fc4(x.view(x.size(0), -1)))
        x = self.fc5(x)
        return x

In [34]:
env.reset()
screen = env.render(mode = 'rgb_array')
print(screen.shape)
time.sleep(1)
env.close()

(400, 600, 3)
